In [1]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg

from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pbar = ProgressBar()
pbar.register()

# Input data

### sa data

In [2]:
%%time
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)
sa_dask = dg.from_geopandas(sa, npartitions=4)

del sa

CPU times: user 177 ms, sys: 20.8 ms, total: 198 ms
Wall time: 551 ms


### pga data

In [3]:
%%time
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))
pga_dask = dg.from_geopandas(pgaGeo, npartitions=4)

del pga
del pgaGeo


CPU times: user 3min 49s, sys: 20.5 s, total: 4min 10s
Wall time: 5min 43s


# Merge SA data with PGA events (dask-geodataframe)

In [4]:
type(pga_dask); type(sa_dask)

dask_geopandas.core.GeoDataFrame

dask_geopandas.core.GeoDataFrame

In [5]:
def dask_sjoin(dd1, dd2):
    join = gpd.sjoin(dd1, dd2, op="within")
    return join

In [6]:
%%time
PGA_SA = dd.map_partitions(dask_sjoin, pga_dask, sa_dask)
# PGA_SA = gpd.sjoin(pga_dask.compute(), sa_dask.compute(), )#, op="within")

CPU times: user 180 ms, sys: 89.8 ms, total: 270 ms
Wall time: 728 ms


In [8]:
PGA_SA.to_parquet('/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_SA.parquet')